In [ ]:
import ir_datasets
import pandas as pd
import re
import json
import os
import html
import unicodedata
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk

# تحميل موارد NLTK إذا لم تكن محمّلة
nltk.download('stopwords')

# 1. تحميل مجموعة البيانات
dataset = ir_datasets.load("msmarco-passage/train")

# 2. قراءة 200 ألف وثيقة وتحويلها إلى قائمة من القواميس
docs_data = []
count = 0
LIMIT = 500_000
seen_ids = set()  # لتتبع المعرّفات التي تمت رؤيتها

# 3. إصلاح النصوص المشوهة
def safe_text(text):
    try:
        return text.encode('latin1').decode('utf-8')
    except:
        return text

# 4. تنظيف النصوص بشكل احترافي
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

def advanced_preprocess(text):
    # فك ترميز HTML مثل &amp; و &quot;
    text = html.unescape(text)
    
    # إزالة الحروف الغير قابلة للطباعة
    text = ''.join(c for c in text if c.isprintable())

    # تحويل النص إلى حروف صغيرة
    text = text.lower()

    # إزالة الروابط
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # إزالة الإيميلات
    text = re.sub(r'\S+@\S+', '', text)

    # إزالة علامات HTML
    text = re.sub(r'<.*?>', '', text)

    # إزالة الرموز والرقم المفرد
    text = re.sub(r'[^a-z\s]', ' ', text)

    # إزالة التكرارات الزائدة (مثل cooool → cool)
    text = re.sub(r'(.)\1{2,}', r'\1', text)

    # إزالة الفراغات الزائدة
    text = re.sub(r'\s+', ' ', text).strip()

    # إزالة الكلمات الشائعة والكلمات القصيرة
    words = [stemmer.stem(w) for w in text.split() if w not in stop_words and len(w) > 2]

    return ' '.join(words)

# 5. تحميل الوثائق وتنظيفها
doc_iterator = dataset.docs_iter()

while count < LIMIT:
    try:
        doc = next(doc_iterator)

        # تجاهل التكرارات
        if doc.doc_id in seen_ids:
            continue
        seen_ids.add(doc.doc_id)

        text_clean = safe_text(doc.text)

        docs_data.append({
            "id": doc.doc_id,
            "text": text_clean
        })
        count += 1

        if count % 10000 == 0:
            print(f"✅ تم تحميل {count} وثيقة...")

    except Exception:
        continue

# 6. إنشاء DataFrame
df = pd.DataFrame(docs_data)

# 7. تنظيف النصوص المتقدمة
print("\n🧹 بدء تنظيف النصوص...")
df['clean_text'] = df['text'].apply(advanced_preprocess)

# 8. إزالة أي تكرارات محتملة (كإجراء إضافي)
df = df.drop_duplicates(subset='id', keep='first')

# 9. حفظ البيانات
output_dir = r"..\data\msmarco_train\processed"
output_file = os.path.join(output_dir, "processed_msmarco_train.json")
os.makedirs(output_dir, exist_ok=True)

df[['id', 'clean_text']].to_json(output_file, orient='records', lines=True, force_ascii=False)

print(f"\n📁 تم حفظ البيانات بنجاح في الملف:\n{output_file}")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✅ تم تحميل 10000 وثيقة...
✅ تم تحميل 20000 وثيقة...
✅ تم تحميل 30000 وثيقة...
✅ تم تحميل 40000 وثيقة...
✅ تم تحميل 50000 وثيقة...
✅ تم تحميل 60000 وثيقة...
✅ تم تحميل 70000 وثيقة...
✅ تم تحميل 80000 وثيقة...
✅ تم تحميل 90000 وثيقة...
✅ تم تحميل 100000 وثيقة...
✅ تم تحميل 110000 وثيقة...
✅ تم تحميل 120000 وثيقة...
✅ تم تحميل 130000 وثيقة...
✅ تم تحميل 140000 وثيقة...
✅ تم تحميل 150000 وثيقة...
✅ تم تحميل 160000 وثيقة...
✅ تم تحميل 170000 وثيقة...
✅ تم تحميل 180000 وثيقة...
✅ تم تحميل 190000 وثيقة...
✅ تم تحميل 200000 وثيقة...
✅ تم تحميل 210000 وثيقة...
✅ تم تحميل 220000 وثيقة...
✅ تم تحميل 230000 وثيقة...
✅ تم تحميل 240000 وثيقة...
✅ تم تحميل 250000 وثيقة...
✅ تم تحميل 260000 وثيقة...
✅ تم تحميل 270000 وثيقة...
✅ تم تحميل 280000 وثيقة...
✅ تم تحميل 290000 وثيقة...
✅ تم تحميل 300000 وثيقة...
✅ تم تحميل 310000 وثيقة...
✅ تم تحميل 320000 وثيقة...
✅ تم تحميل 330000 وثيقة...
✅ تم تحميل 340000 وثيقة...
✅ تم تحميل 350000 وثيقة...
✅ تم تحميل 360000 وثيقة...
✅ تم تحميل 370000 وثيقة...
✅ تم تحميل

In [18]:
import json
import os
import joblib
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. تحميل البيانات
input_file = r"C:\Users\Mohammad Mihdi\Desktop\Projects\IR\data\msmarco_train\processed\processed_msmarco_train.json"
with open(input_file, 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

# 2. استخراج النصوص والمعرفات
doc_ids = [doc['id'] for doc in data]
texts = [doc['clean_text'] for doc in data]

# 3. استخدام TF-IDF مع دالة التنظيف

import html
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

def advanced_preprocess(text):
    text = html.unescape(text)
    text = ''.join(c for c in text if c.isprintable())
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = [stemmer.stem(w) for w in text.split() if w not in stop_words and len(w) > 2]
    return ' '.join(words)


vectorizer = TfidfVectorizer(
    preprocessor=advanced_preprocess,
    tokenizer=str.split,
    max_df=0.85,
    min_df=2,
    sublinear_tf=True,
    norm='l2'
)

# 4. تطبيق TF-IDF
tfidf_matrix = vectorizer.fit_transform(texts)

# 5. حفظ النتائج
output_dir = r"C:\Users\Mohammad Mihdi\Desktop\Projects\IR\data\msmarco_train\index\TFIDF"
os.makedirs(output_dir, exist_ok=True)

joblib.dump(vectorizer, os.path.join(output_dir, "tfidf_vectorizer_msmarco_train.joblib"))
joblib.dump(tfidf_matrix, os.path.join(output_dir, "tfidf_matrix_msmarco_train.joblib"))
joblib.dump(doc_ids, os.path.join(output_dir, "doc_ids_msmarco_train.joblib"))

print("✅ تم حفظ ملفات TF-IDF بنجاح.")


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


✅ تم حفظ ملفات TF-IDF بنجاح.


In [12]:
import ir_datasets
import pandas as pd
import re
import json
import os
import html
import unicodedata
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk

# تحميل موارد NLTK إذا لم تكن محمّلة
nltk.download('stopwords')

# 1. تحميل مجموعة البيانات
dataset = ir_datasets.load("antique/train")

# 2. قراءة 200 ألف وثيقة وتحويلها إلى قائمة من القواميس
docs_data = []
count = 0
LIMIT = 405_000
seen_ids = set()  # لتتبع المعرّفات التي تمت رؤيتها

# 3. إصلاح النصوص المشوهة
def safe_text(text):
    try:
        return text.encode('latin1').decode('utf-8')
    except:
        return text

# 4. تنظيف النصوص بشكل احترافي
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

def advanced_preprocess(text):
    # فك ترميز HTML مثل &amp; و &quot;
    text = html.unescape(text)
    
    # إزالة الحروف الغير قابلة للطباعة
    text = ''.join(c for c in text if c.isprintable())

    # تحويل النص إلى حروف صغيرة
    text = text.lower()

    # إزالة الروابط
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # إزالة الإيميلات
    text = re.sub(r'\S+@\S+', '', text)

    # إزالة علامات HTML
    text = re.sub(r'<.*?>', '', text)

    # إزالة الرموز والرقم المفرد
    text = re.sub(r'[^a-z\s]', ' ', text)

    # إزالة التكرارات الزائدة (مثل cooool → cool)
    text = re.sub(r'(.)\1{2,}', r'\1', text)

    # إزالة الفراغات الزائدة
    text = re.sub(r'\s+', ' ', text).strip()

    # إزالة الكلمات الشائعة والكلمات القصيرة
    words = [stemmer.stem(w) for w in text.split() if w not in stop_words and len(w) > 2]

    return ' '.join(words)

# 5. تحميل الوثائق وتنظيفها
# 5. تحميل الوثائق وتنظيفها
doc_iterator = dataset.docs_iter()
while count < LIMIT:
    try:
        doc = next(doc_iterator)
    except StopIteration:
        print("📌 انتهت الوثائق الموجودة في الداتا سيت.")
        break
    except Exception:
        continue  # تجاوز أي خطأ غير متوقع

    if doc.doc_id in seen_ids:
        continue

    seen_ids.add(doc.doc_id)
    text_clean = safe_text(doc.text)
    docs_data.append({
        "id": doc.doc_id,
        "text": text_clean
    })
    count += 1

    if count % 10000 == 0:
        print(f"✅ تم تحميل {count} وثيقة...")


# 6. إنشاء DataFrame
df = pd.DataFrame(docs_data)

# 7. تنظيف النصوص المتقدمة
print("\n🧹 بدء تنظيف النصوص...")
df['clean_text'] = df['text'].apply(advanced_preprocess)

# 8. إزالة أي تكرارات محتملة (كإجراء إضافي)
df = df.drop_duplicates(subset='id', keep='first')

# 9. حفظ البيانات
output_dir = r"..\data\antique_train\processed"
output_file = os.path.join(output_dir, "processed_antique_train.json")
os.makedirs(output_dir, exist_ok=True)

df[['id', 'clean_text']].to_json(output_file, orient='records', lines=True, force_ascii=False)

print(f"\n📁 تم حفظ البيانات بنجاح في الملف:\n{output_file}")

[nltk_data] Downloading package stopwords to C:\Users\Mohammad
[nltk_data]     Mihdi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✅ تم تحميل 10000 وثيقة...
✅ تم تحميل 20000 وثيقة...
✅ تم تحميل 30000 وثيقة...
✅ تم تحميل 40000 وثيقة...
✅ تم تحميل 50000 وثيقة...
✅ تم تحميل 60000 وثيقة...
✅ تم تحميل 70000 وثيقة...
✅ تم تحميل 80000 وثيقة...
✅ تم تحميل 90000 وثيقة...
✅ تم تحميل 100000 وثيقة...
✅ تم تحميل 110000 وثيقة...
✅ تم تحميل 120000 وثيقة...
✅ تم تحميل 130000 وثيقة...
✅ تم تحميل 140000 وثيقة...
✅ تم تحميل 150000 وثيقة...
✅ تم تحميل 160000 وثيقة...
✅ تم تحميل 170000 وثيقة...
✅ تم تحميل 180000 وثيقة...
✅ تم تحميل 190000 وثيقة...
✅ تم تحميل 200000 وثيقة...
✅ تم تحميل 210000 وثيقة...
✅ تم تحميل 220000 وثيقة...
✅ تم تحميل 230000 وثيقة...
✅ تم تحميل 240000 وثيقة...
✅ تم تحميل 250000 وثيقة...
✅ تم تحميل 260000 وثيقة...
✅ تم تحميل 270000 وثيقة...
✅ تم تحميل 280000 وثيقة...
✅ تم تحميل 290000 وثيقة...
✅ تم تحميل 300000 وثيقة...
✅ تم تحميل 310000 وثيقة...
✅ تم تحميل 320000 وثيقة...
✅ تم تحميل 330000 وثيقة...
✅ تم تحميل 340000 وثيقة...
✅ تم تحميل 350000 وثيقة...
✅ تم تحميل 360000 وثيقة...
✅ تم تحميل 370000 وثيقة...
📌 انتهت ال

In [13]:
import json
import os
import joblib
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. تحميل البيانات
input_file = r"..\data\antique_train\processed\processed_antique_train.json"
with open(input_file, 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

# 2. استخراج النصوص والمعرفات
doc_ids = [doc['id'] for doc in data]
texts = [doc['clean_text'] for doc in data]

# 3. استخدام TF-IDF مع دالة التنظيف

import html
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

def advanced_preprocess(text):
    text = html.unescape(text)
    text = ''.join(c for c in text if c.isprintable())
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = [stemmer.stem(w) for w in text.split() if w not in stop_words and len(w) > 2]
    return ' '.join(words)


vectorizer = TfidfVectorizer(
    preprocessor=advanced_preprocess,
    tokenizer=str.split,
    max_df=0.85,
    min_df=2,
    sublinear_tf=True,
    norm='l2'
)

# 4. تطبيق TF-IDF
tfidf_matrix = vectorizer.fit_transform(texts)

# 5. حفظ النتائج
output_dir = r"..\data\antique_train\index\TFIDF"
os.makedirs(output_dir, exist_ok=True)

joblib.dump(vectorizer, os.path.join(output_dir, "tfidf_vectorizer_antique_train.joblib"))
joblib.dump(tfidf_matrix, os.path.join(output_dir, "tfidf_matrix_antique_train.joblib"))
joblib.dump(doc_ids, os.path.join(output_dir, "doc_ids_antique_train.joblib"))

print("✅ تم حفظ ملفات TF-IDF بنجاح.")


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


✅ تم حفظ ملفات TF-IDF بنجاح.


In [1]:
import json
import os
import joblib
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
import re

sys.path.append("..")

from services.documents_service import preprocess

# 1. تحميل البيانات
input_file = r"..\data\beir_quora_test\processed\processed_beir_quora_test.json"
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 2. استخراج النصوص والمعرفات
doc_ids = [doc['doc_id'] for doc in data]
texts = [" ".join(doc['tokens']) for doc in data]  # نصوص نصية

# 3. إعداد TfidfVectorizer مع preprocess كـ tokenizer
vectorizer = TfidfVectorizer(
    tokenizer=preprocess,   # دالة تقسيم الكلمات مع التنظيف
    preprocessor=None,      # لا تستخدم preprocessor آخر
    max_df=0.85,
    min_df=2,
    sublinear_tf=True,
    norm='l2',
    lowercase=False         # لأن preprocess تقوم بتحويل الحروف إلى صغيرة بالفعل
)

# 4. تطبيق TF-IDF
tfidf_matrix = vectorizer.fit_transform(texts)

# 5. حفظ النتائج
output_dir = r"..\data\beir_quora_test\index\TFIDF"
os.makedirs(output_dir, exist_ok=True)

joblib.dump(vectorizer, os.path.join(output_dir, "tfidf_vectorizer_beir_quora_test.joblib"))
joblib.dump(tfidf_matrix, os.path.join(output_dir, "tfidf_matrix_beir_quora_test.joblib"))
joblib.dump(doc_ids, os.path.join(output_dir, "doc_ids_beir_quora_test.joblib"))

print("✅ تم حفظ ملفات TF-IDF بنجاح.")

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


✅ تم حفظ ملفات TF-IDF بنجاح.
